In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#def load_data():
global smart
global gas
global weather
df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
smart = df.iloc[:,:7]
gas = df.iloc[:, 7:]

In [3]:
#load_data()

# Change all datetime columns to the same column name
smart = smart.rename(index=str, columns={"Timestamp":"datetime"})
gas = gas.rename(index=str, columns={"gasTimestamp":"datetime"})

#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter', 'T', 'Q']]
#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter']]
smart.head()

# Set datetime as index
smart = smart.set_index(['datetime'])
gas = gas.set_index(['datetime'])

In [4]:
"""
Resample the dataframes to 10s intervals.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
"""
smart = smart.resample('10s').mean()
gas = gas.resample('H').mean() # Does not look nice, but it works

In [5]:
smart.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
datetime,,,,,,
2017-03-07 18:14:20,3717.472,0.0,3883.382,0.0,376.0,0.0
2017-03-07 18:14:30,3717.473,0.0,3883.382,0.0,381.0,0.0
2017-03-07 18:14:40,3717.474,0.0,3883.382,0.0,378.0,0.0
2017-03-07 18:14:50,3717.475,0.0,3883.382,0.0,376.0,0.0
2017-03-07 18:15:00,3717.476,0.0,3883.382,0.0,380.0,0.0


In [6]:
gas.head()

,gasMeter
datetime,
2017-03-07 18:00:00,1599.792
2017-03-07 19:00:00,1599.986
2017-03-07 20:00:00,1600.179
2017-03-07 21:00:00,1600.367
2017-03-07 22:00:00,1600.513


# Working with electricity dataframe

In [7]:
#First check how many nans are there
smart_nan=smart.isnull().any(axis=1)
print(smart[smart_nan].info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2947 entries, 2017-03-07 18:16:10 to 2017-03-28 23:50:30
Data columns (total 6 columns):
eMeter             0 non-null float64
eMeterReturn       0 non-null float64
eMeterLow          0 non-null float64
eMeterLowReturn    0 non-null float64
ePower             0 non-null float64
ePowerReturn       0 non-null float64
dtypes: float64(6)
memory usage: 161.2 KB
None


In [8]:
smart.describe()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
count,180567.000000,180567.0,180567.000000,180567.0,180567.000000,180567.0
mean,3772.187364,0.0,3943.679690,0.0,459.004615,0.0
std,32.169904,0.0,36.106887,0.0,452.420914,0.0
min,3717.472000,0.0,3883.382000,0.0,238.000000,0.0
25%,3737.922000,0.0,3918.407500,0.0,319.000000,0.0
50%,3775.712000,0.0,3939.401000,0.0,351.000000,0.0
75%,3801.757500,0.0,3973.161000,0.0,414.000000,0.0
max,3830.551000,0.0,4004.212000,0.0,7765.000000,0.0


In [9]:
print(smart[smart_nan])

                     eMeter  eMeterReturn  eMeterLow  eMeterLowReturn  ePower  \
datetime                                                                        
2017-03-07 18:16:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:17:40     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:46:00     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:47:20     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:17:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:18:30     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:48:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:49:30     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:18:00     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:19:20     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:49:10     NaN 

In [10]:
#Then just interpolate
#smart_final=smart.interpolate('time')
#print(smart_final)

In [14]:
smart_nan=smart_nan*1

#print(smart_nan)
temp=smart_nan.groupby((smart_nan != smart_nan.shift()).cumsum()).transform('size') * smart_nan
print(temp)

datetime
2017-03-07 18:14:20    0
2017-03-07 18:14:30    0
2017-03-07 18:14:40    0
2017-03-07 18:14:50    0
2017-03-07 18:15:00    0
2017-03-07 18:15:10    0
2017-03-07 18:15:20    0
2017-03-07 18:15:30    0
2017-03-07 18:15:40    0
2017-03-07 18:15:50    0
2017-03-07 18:16:00    0
2017-03-07 18:16:10    1
2017-03-07 18:16:20    0
2017-03-07 18:16:30    0
2017-03-07 18:16:40    0
2017-03-07 18:16:50    0
2017-03-07 18:17:00    0
2017-03-07 18:17:10    0
2017-03-07 18:17:20    0
2017-03-07 18:17:30    0
2017-03-07 18:17:40    1
2017-03-07 18:17:50    0
2017-03-07 18:18:00    0
2017-03-07 18:18:10    0
2017-03-07 18:18:20    0
2017-03-07 18:18:30    0
2017-03-07 18:18:40    0
2017-03-07 18:18:50    0
2017-03-07 18:19:00    0
2017-03-07 18:19:10    0
                      ..
2017-03-28 23:55:00    0
2017-03-28 23:55:10    0
2017-03-28 23:55:20    0
2017-03-28 23:55:30    0
2017-03-28 23:55:40    0
2017-03-28 23:55:50    0
2017-03-28 23:56:00    0
2017-03-28 23:56:10    0
2017-03-28 23:56

<bound method NDFrame.describe of datetime
2017-03-07 18:14:20    0
2017-03-07 18:14:30    0
2017-03-07 18:14:40    0
2017-03-07 18:14:50    0
2017-03-07 18:15:00    0
2017-03-07 18:15:10    0
2017-03-07 18:15:20    0
2017-03-07 18:15:30    0
2017-03-07 18:15:40    0
2017-03-07 18:15:50    0
2017-03-07 18:16:00    0
2017-03-07 18:16:10    1
2017-03-07 18:16:20    0
2017-03-07 18:16:30    0
2017-03-07 18:16:40    0
2017-03-07 18:16:50    0
2017-03-07 18:17:00    0
2017-03-07 18:17:10    0
2017-03-07 18:17:20    0
2017-03-07 18:17:30    0
2017-03-07 18:17:40    1
2017-03-07 18:17:50    0
2017-03-07 18:18:00    0
2017-03-07 18:18:10    0
2017-03-07 18:18:20    0
2017-03-07 18:18:30    0
2017-03-07 18:18:40    0
2017-03-07 18:18:50    0
2017-03-07 18:19:00    0
2017-03-07 18:19:10    0
                      ..
2017-03-28 23:55:00    0
2017-03-28 23:55:10    0
2017-03-28 23:55:20    0
2017-03-28 23:55:30    0
2017-03-28 23:55:40    0
2017-03-28 23:55:50    0
2017-03-28 23:56:00    0
2017-03

In [17]:
temp.describe()
temp.to_csv('~/temp.csv')

In [22]:
smart.loc['2017-03-26 03:00:00']

eMeter             3811.654
eMeterReturn          0.000
eMeterLow          3988.892
eMeterLowReturn       0.000
ePower              334.000
ePowerReturn          0.000
Name: 2017-03-26 03:00:00, dtype: float64

In [ ]:
#Delete rows with NaN in 'ePower' column
smart_final=smart.dropna(subset=['ePower'])
smart_final.isnull().sum()

# Working with gas dataframe

In [ ]:
#Inspect for NaNs
gas_nan=gas.isnull().any(axis=1)
print(gas[gas_nan])

In [ ]:
#Interpolate
gas_final = gas.resample('10s').interpolate(method='time')
# Create a ''gasPower'' column. This equals the gas used in that hour
gas_final['gasPower'] = gas_final['gasMeter'].diff()
# First entry is NaN, replace NaN by second entry
gas_final['gasPower'][0] = gas_final['gasPower'][1]
print(gas_final)

In [ ]:
#Fit datetimes
#gas_selected=gas_final.ix[smart_final.index[0]:gas_final.index[-1]]
#print(gas_selected)

In [ ]:
# Gas datetime ends at 23:00:00 (at least in this specific file) and Electricity date time ends at 23:59:50 so
# when merging final DateTime will be 23:00:00
df = pd.merge(smart_final,gas_final,left_index=True, right_index=True)

In [ ]:
print(df)